### Importing packages

In [ ]:
#!/usr/bin/env python
import os, sys, csv
from datetime import datetime
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from collections import OrderedDict, defaultdict, Counter
import shutil, argparse, re
from heapq import nlargest
from operator import itemgetter
import numpy as np
import matplotlib.pyplot as plt
import operator, statistics
from string import ascii_uppercase
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

TOPN = 10
path = "/Users/kyle/Desktop/"
NEW = '\n'
TAB = '\t'

### Parsing command line arguments

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('-i', '--input', nargs='+', help='name of sequencing reads file', required=True)
parser.add_argument('-o', '--output',  help='name and pathway to output file', default=r'dFASTR_output ' + datetime.now().strftime(" %m-%d-%Y") + '.csv')
parser.add_argument('-ha', '--histall', help='output all (both new and old) histograms')
parser.add_argument('-nh', '--new_hist', help='output new histograms')
parser.add_argument('-oh', '--old_hist', help='output old histograms')

args = parser.parse_args()

filenamelist = args.input

In [ ]:
class Vividict(dict):
    def __missing__(self, key):
        value = self[key] = type(self)()
        return value

def getloci(path,flist):
    """Find the loci from the headers of all FASTA files, return as list."""
    llist = []
    for f in flist:  #in args.input:
        for read in SeqIO.parse(path+f, 'fasta'):
            llist.append(re.split('\W',read.id)[0])
    return list(set(llist))

def findlocus(header,l):
    """Find the locus name in the header of a FASTA record, return as string."""
    templist = [x for x in l if x in header]
    return sorted(templist, key=len)[-1]

def findsize(header):
    """Find the size (=frequency) of sequence in the header of a FASTA record, return as integer."""
    sizelong = [word for word in header.split(';') if any(x in word for x in ['size','freq'])][0]
    return int(sizelong.split('=')[-1])

def topNsequences(d, N=12):
    """Find the top N most frequent sequences from a dictionary of all sequences, return as dictionary."""
    topNtuples = sorted(d.items(), key=itemgetter(1), reverse=True)[:N]
    return dict(topNtuples)

def lengthsofseqalleles(s):
    """Find the lengths of alleles from list of sequences s, return as list."""
    return [len(l) for l in s]

def rulebasedcall_1(d, r=2): 
    """Use a rule-based logic to call alleles.
    d=dictionary of TOPN sequences: d[seq]=size
    """
    c = list() # c = list of candidate alleles
    allele_1 = max(d, key=lambda key: d[key])
    topfrequency = d.pop(allele_1)
    if topfrequency >= 10:
        c.append(allele_1)
        for read in d:
            if d[read] >= 10 and len(read) > 100:
                #if (len(allele_1) -  len(read)) % r == 0:
                if len(read) >= len(allele_1):
                    if d[read] >= (topfrequency * 0.3):
                        c.append(read)
                if len(read) < len(allele_1):
                    if d[read] >= (topfrequency * 0.3):
                        c.append(read)
    if len(c)==1:
        c = c*2
    return c

def lengths2consider(d):
    """Find the lengths (alleles) to consider from a dictionary of top N sequences.
    d=dictionary of TOPN sequences: d[seq]=size
    RETURNS: a list of the lengths (allels), excluding any outliers
    """
    lengthlist = list(set([len(seq) for seq in d]))
    median = statistics.median(lengthlist) # m = median (about)
    return sorted([ x for x in lengthlist if 0.5*median <= x <= 2*median ])
     
def preplengthdictionary(flist, llist):
    """Populate a defaultdict to be used in collecting length-to-frequency data.
    RETURNS: dictionary with intended use: length2freq[locus][filename][len] += int """
    l2f = Vividict()
    for f in flist:  #in args.input:
        for locus in llist:
            l2f[locus][f] = defaultdict(int)
    return l2f

def collectData(path, flist, l2f, llist):
    """Collect sequence, size(frequency), length, and locus data from FASTA files in list of filenames.
    RETURNS: dictionaries with intended use: length2freq[locus][filename][len] = freq   where freq is integer
    RETURNS: dictionaries with intended use: seq2freq[locus][filename][seq] = freq """
    s2f = Vividict()
    for f in flist:  #in args.input:
        for read in SeqIO.parse(path+f, 'fasta'):
            locus = findlocus(read.id,llist)
            size = findsize(read.id)
            length = len(read.seq)
            seq = str(read.seq)
            if size > 1:
                s2f[locus][f][seq] = size
                l2f[locus][f][length] += size
    return s2f, l2f

def collectFrequencyArrays(path, flist, llist, MAXLENGTH=1000):
    """Collect sequence, size(frequency), length, and locus data from FASTA files in list of filenames.
    RETURNS: dictionaries with intended use: length2freq[locus][filename][len] = freq   where freq is integer
    RETURNS: dictionaries with intended use: seq2freq[locus][filename][seq] = freq """
    lengtharray = Vividict()
    for f in flist:  #in args.input:
        for locus in llist:
            lengtharray[locus][f] = np.zeros(MAXLENGTH, dtype=int)
        for read in SeqIO.parse(path+f, 'fasta'):
            locus = findlocus(read.id,llist)
            size = findsize(read.id)
            length = len(read.seq)
            seq = str(read.seq)
            if size > 1:
                lengtharray[locus][f][length] += size
    return lengtharray

######## This is where the program starts ###############           

#1. Make list of loci in file
print("Finding loci...", end="")
loci_list = getloci(path,filenamelist)

#2a. Prep frequency arrays
print("Collecting length frequencies...", end="")
lengtharray = collectFrequencyArrays(path, filenamelist, loci_list)

def displayhistograms(lengtharray, MINREADS=20):

    """Plot histograms of read length frequencies.
    INPUT: lengtharray = dictionary that maps locus and filename to an array of read frequencies:
            lengtharray[locus][filename] = array([0,0,0,0,...0,0])
    RETURNS: dictionaries with intended use: seq2freq[locus][filename][seq] = freq """

    for locus in lengtharray: 
        plotn = 1
        plt.figure()
        for filename in lengtharray[locus]:
            plt.title("Heterozygous", fontsize=20, fontweight='bold')
            plt.xlabel("Read/allele length (bp)", fontsize=20, fontweight='bold')
            plt.ylabel("Read frequency", fontsize=20, fontweight='bold')
            idx = np.nonzero(lengtharray[locus][filename]>=MINREADS) # idx = indices of read frequencies >= MINREADS
            ax = plt.bar(idx[0],lengtharray[locus][filename][idx])
            plt.xticks(idx[0], rotation=30, fontsize=13, fontweight='bold')
            plt.yticks(fontweight='bold', fontsize=13)
            plotn +=1
        plt.show()
        # plt.savefig('foo.png')
# displayhistograms(lengtharray)

#2. Prep length2freq dict
print("prepping length2freq dictionary...", end="")
length2freq = preplengthdictionary(filenamelist,loci_list) #intended use: length2freq[locus][filename][len] += int

#3. Populate master data dictionaries
print("collecting data...", end="")
seq2freq, length2freq = collectData(path, filenamelist, length2freq, loci_list) 

def printoldschoolhistograms(lengtharray, seq2freq, MINREADS=20):
    """Plot histograms of read length frequencies.
    INPUT: lengtharray = dictionary that maps locus and filename to an array of read frequencies:
            lengtharray[locus][filename] = array([0,0,0,0,...0,0])
    RETURNS: dictionaries with intended use: seq2freq[locus][filename][seq] = freq """
    for locus in lengtharray:  #in args.input:
        print(".", end="")
        for filename in lengtharray[locus]:
            with open(path+filename+locus+'histograms.txt','w') as f:
                f.write('-'*100+NEW)
                f.write('$locus = ' + locus + NEW)
            # for filename in lengtharray[locus]:
                print(".", end="")
                f.write('~'*100+NEW)
                f.write('~'*100+NEW)
                f.write('#sample = ' + filename + NEW)
                idx = np.nonzero(lengtharray[locus][filename]>=MINREADS) # idx = indices of read frequencies >= MINREADS
                maxreads = max(lengtharray[locus][filename])
                for i in range(np.amin(idx)-10,np.amax(idx)+10):
                    nstars = int((lengtharray[locus][filename][i]*100)/maxreads)
                    f.write(" ".join([locus, filename, str(i).rjust(3), '*'*nstars])+NEW)
                    topNdict = topNsequences(seq2freq[locus][filename])
                rank = 0
                longestallele = max(lengthsofseqalleles(topNdict.keys()))
                for record in topNdict:
                    length = str(len(record)).ljust(3)
                    freq = ('freq='+str(topNdict[record])).ljust(12)
                    sequence = record.ljust(longestallele+3)
                    srank = str(rank).rjust(2)
                    f.write(" ".join(['@',locus, filename, srank, freq, length, '()', sequence, length, srank, NEW]))
                    rank += 1

print("Writing histograms...", end="")

printoldschoolhistograms(lengtharray, seq2freq)

def namealleles(genotypedictionary):
    allelenames = Vividict() # Use as allelenames[locus][seq]=allelename
    for locus in genotypedictionary:
        templist = [] # a temporary list of all alleles for a particular locus
        for sample in genotypedictionary[locus]:
            for allele in genotypedictionary[locus][sample]:
                templist.append(allele)
        templist = list(set(templist)) # uniquify list
        templendict = defaultdict(list)
        for seq in templist:
            templendict[len(seq)].append(seq)
        lcounter = 0
        for length in templendict:
            templendict[length].sort()
            for seq in templendict[length]:
                allelename = '_'.join([locus,str(length),ascii_uppercase[lcounter]])
                allelenames[locus][seq] = allelename
                lcounter += 1
                print(locus, length, seq, allelenames[locus][seq])
    return allelenames

def outputallelenames(allelenamedict, path, alleleoutfilename): #output alleles to fasta file
    seqlist = []
    for locus in allelenames:
        for alleleseq in allelenames[locus]:
            header = allelenames[locus][alleleseq]
            seqlist.append(SeqRecord(Seq(alleleseq),id=header, description=header))
    with open(path+alleleoutfilename, 'w') as handle:
        SeqIO.write(seqlist, handle, 'fasta')

# allelenames = namealleles(oldschoolcalls)
# outputallelenames(allelenames, path, 'allelenames.txt')

#4. Call alleles          
print("calling alleles...", end="")

candidatealleles = Vividict() # intended use: length2freq[locus][filename][method] = [allele1,allele2]

for locus in seq2freq:  #in args.input:
    for filename in seq2freq[locus]:
        topNdict = topNsequences(seq2freq[locus][filename])
        topNlengths = lengths2consider(topNdict)

        candidatealleles[locus][filename]['rulebasedcall_1'] = rulebasedcall_1(topNdict)

def displaycalls(candidatealleles):
    """Plot histograms of read length frequencies.
    INPUT: candidatealleles = dictionary that maps locus and filename to an array of read frequencies:
            candidatealleles[locus][filename][callname] = [allele_1, allele_2]"""
    print("",)
    print("{:<20} {:<20} {:<20}".format('Filename','Locus','Alleles'))
    for locus in candidatealleles:  #in args.input:
        for filename in candidatealleles[locus]:
            a = lengthsofseqalleles(candidatealleles[locus][filename]['Alleles'])         
            print("{:<20} {:<20} {:<20}".format(filename, locus, str(a)))

# displaycalls(candidatealleles)

print('done')
